In [3]:
import math
areas = [0.277834,
         0.133400,
         0.088265,
         0.334002,
         0.111334,
         0.033099,
         0.022066]

areas = [math.sqrt(x) for x in areas]
full = sum(areas)
[z/full for z in areas]

[0.2167789213723987,
 0.15021118424640953,
 0.1221852039373189,
 0.23768342059958592,
 0.13722658686508193,
 0.07482242708642484,
 0.06109225589277995]

In [2]:
import geopandas as gpd
from gen_planner.python.src.genplanner import GenPlanner
from func_zones import func_zones, basic_scenario, gen_plan

res = GenPlanner(gpd.read_file('1.gpkg')).terr2district2zone2block(gen_plan)
res.explore(column='func_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)

In [4]:
full = res.area.sum()
res['area'] = res.area
areas = res.groupby('func_zone')['area'].sum().reset_index()
areas['percent'] = areas['area'] / full
areas

,func_zone,area,percent
0,agriculture territory,1.435793e+07,0.021025
1,business territory,7.276455e+07,0.106552
2,industrial territory,8.192722e+07,0.119969
3,recreation territory,2.300777e+08,0.336911
4,residential territory,2.002020e+08,0.293163
5,special territory,1.599537e+07,0.023423
6,transport territory,6.757800e+07,0.098957


In [ ]:

res.explore(column='func_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)

In [ ]:
from gen_planner.python.src.genplanner import GenPlanner
from func_zones import func_zones, basic_scenario
import geopandas as gpd

res = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).district2zone2block(basic_scenario)

res.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2')

In [ ]:
res = GenPlanner(gpd.read_file('Аэродром_лодейнопольское_поселение.geojson')).district2zone2block(basic_scenario)
res.explore(column='func_zone', tiles='CartoDB positron', cmap='Dark2')

In [ ]:
import geopandas as gpd
from gen_planner.python.src.genplanner import GenPlanner
from func_zones import func_zones, basic_scenario

res = GenPlanner(gpd.read_file('светогорск.geojson').buffer(0.1)).district2zone2block(basic_scenario)
res.explore(column='func_zone', tiles='CartoDB positron', cmap='Dark2')